# Mini Proyecto

datos: https://www.cs.cmu.edu/~ark/personas/

In [2]:
# Librerias

import pandas as pd
import pyspark
# Cargar Funciones Extra
from pyspark.sql.functions import * 
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Proyecto_1').getOrCreate()

22/02/26 23:21:01 WARN Utils: Your hostname, MacBook-Air-de-Daniel.local resolves to a loopback address: 127.0.0.1; using 192.168.68.104 instead (on interface en0)
22/02/26 23:21:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/26 23:21:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [27]:
#Carga de bibliotecas extras
import numpy as np
import matplotlib.pyplot as plt

#Carga de NLTK
import nltk
import nltk
nltk.download('stopwords')
import string

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/danielfragoso/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
def read_tsv(direccion):
    import csv
    tsv_file = open(direccion)
    read_tsv = csv.reader(tsv_file, delimiter= "\t")
    lista = [row for row in read_tsv]
    tsv_file.close()
    return(spark.sparkContext.parallelize(lista))

def read_txt(direccion):
    import csv
    txt_file = open(direccion)
    read_tsv = csv.reader(txt_file, delimiter= "\t")
    lista = [row for row in read_tsv]
    txt_file.close()
    return(spark.sparkContext.parallelize(lista))

In [5]:
# Importamos datos:

character = read_tsv('MovieSummaries/character.metadata.tsv')
movie = read_tsv('MovieSummaries/movie.metadata.tsv')

In [6]:
name = read_txt('MovieSummaries/name.clusters.txt')
tvtropes = read_txt('MovieSummaries/tvtropes.clusters.txt')
plot_summaries =read_txt('MovieSummaries/plot_summaries.txt')

¿Cual es género más visto?

In [7]:
def sep_gen(renglon):
    renglon_separado = renglon[8]
    import re
    generos = re.findall(r'"([\w\s]+)"', renglon_separado)
    for g in generos:
        yield g

In [8]:
genere = movie.flatMap(sep_gen)
genere = genere.map(lambda renglon: (renglon, 1))
# Conteo
genere_count = genere.reduceByKey(lambda val_1 , val_2: val_1 + val_2).sortBy(lambda renglon: renglon[1], ascending = False)

22/02/26 23:21:23 WARN TaskSetManager: Stage 0 contains a task of very large size (4397 KiB). The maximum recommended task size is 1000 KiB.


In [9]:
genere_count.take(10)

[('Drama', 34007),
 ('Comedy', 16349),
 ('Romance Film', 10234),
 ('Action', 8798),
 ('Thriller', 8744),
 ('Short Film', 8141),
 ('World cinema', 7155),
 ('Crime Fiction', 6948),
 ('Indie', 6897),
 ('Documentary', 5630)]

La palabra más larga de las descripciones

In [89]:
def com_string(x, y):
    if len(x) > len(y):
        return x
    return y

def del_url(renglon):
    import re
    generos = re.sub(r'http\S+', ' ', renglon)
    return generos

In [90]:
desc = plot_summaries.map(lambda x: x[1])
desc = desc.map(del_url)

In [91]:
# Reduzcammos a palabras

words = desc.map(lambda x: x.split(' '))

#Eliminamos signos de puntuación y espacios en blanco
import string
def removePunctuationsFunct(x):
    list_punct=list(string.punctuation)
    filtered = [''.join(c for c in s if c not in list_punct) for s in x] 
    filtered_space = [s for s in filtered if s] #remove empty space 
    return filtered

#Removemos stopwords (palabras vacias)
def removeStopWordsFunct(x):
    from nltk.corpus import stopwords
    stop_words=set(stopwords.words('english'))
    filteredSentence = [w for w in x if not w in stop_words]
    return filteredSentence

words = words.map(removeStopWordsFunct)
words = words.map(removePunctuationsFunct)


In [92]:
def sep(renglon):
    for g in renglon:
        return g

words_sep = words.map(sep)

In [93]:
word_legth = words_sep.reduce(com_string)

22/02/27 16:41:31 WARN TaskSetManager: Stage 42 contains a task of very large size (18273 KiB). The maximum recommended task size is 1000 KiB.


In [94]:
word_legth

'PlotdatePrologueThursdayMondaySaturdayTuesdayFridaySundayWednesdayEpilogue'

Contar el número de documentos en los que ocurre cada palabra, bigrama y trigrama.

In [126]:
cont = plot_summaries.map(lambda x: (x[0], x[1].split(' ')))
cont = cont.map(lambda x: (x[0], removeStopWordsFunct(x[1])))
cont = cont.map(lambda x: (x[0], removePunctuationsFunct(x[1])))

In [128]:
cont_war = cont.map(lambda x: (x[0], x[1].count('war')))

In [130]:
cont_war = cont_war.sortBy(lambda renglon: renglon[1], ascending = False)

22/02/27 18:55:44 WARN TaskSetManager: Stage 63 contains a task of very large size (18273 KiB). The maximum recommended task size is 1000 KiB.
22/02/27 18:56:21 WARN TaskSetManager: Stage 64 contains a task of very large size (18273 KiB). The maximum recommended task size is 1000 KiB.


In [131]:
cont_war.take(5)

22/02/27 18:56:57 WARN TaskSetManager: Stage 65 contains a task of very large size (18273 KiB). The maximum recommended task size is 1000 KiB.


[('886892', 10),
 ('3843721', 9),
 ('3653120', 8),
 ('3903364', 8),
 ('1432805', 8)]

In [156]:
war_movies = cont_war.filter(lambda renglon: renglon[1] > 5)

¿Cual es la película que más menciona guerra en su descripción?

In [132]:
movie.filter(lambda x: x[0] == '886892').take(1)

22/02/27 18:58:15 WARN TaskSetManager: Stage 67 contains a task of very large size (4397 KiB). The maximum recommended task size is 1000 KiB.


[['886892',
  '/m/03lv4x',
  'Breaker Morant',
  '1980',
  '4735000',
  '107.0',
  '{"/m/02h40lc": "English Language"}',
  '{"/m/0chghy": "Australia", "/m/0ctw_b": "New Zealand"}',
  '{"/m/01fc50": "Anti-war", "/m/0hj3mx2": "British Empire Film", "/m/03g3w": "History", "/m/04xvlr": "Period piece", "/m/07s9rl0": "Drama", "/m/082gq": "War film", "/m/07k67c": "Anti-war film", "/m/05bh16v": "Courtroom Drama"}']]

<img src='MovieSummaries/Breaker_morant.png'>

¿De que país provienen principalmente las películas de guerra?

In [157]:
war_movies.take(5)

[('886892', 10),
 ('3843721', 9),
 ('3653120', 8),
 ('3903364', 8),
 ('1432805', 8)]

In [160]:
country = movie.map(lambda x: (x[0], x[7]))
country.take(1)

22/02/27 21:57:15 WARN TaskSetManager: Stage 95 contains a task of very large size (4397 KiB). The maximum recommended task size is 1000 KiB.


[('975900', '{"/m/09c7w0": "United States of America"}')]

In [229]:
country_war = country.join(war_movies)

In [230]:
def sep_gen_2(renglon):
    renglon_separado = renglon[0]
    import re
    generos = re.findall(r'"([\w\s]+)"', renglon_separado)
    for g in generos:
        yield g

In [233]:
country_war = country_war.flatMap(lambda x: sep_gen_2(x[1]))

In [237]:
country_war_cont = country_war.map(lambda x: (x, 1)).reduceByKey(lambda val_1 , val_2: val_1 + val_2).sortBy(lambda renglon: renglon[1], ascending = False)

In [238]:
country_war_cont.take(10)

[('United States of America', 12),
 ('United Kingdom', 6),
 ('Japan', 4),
 ('Canada', 2),
 ('Serbia', 2),
 ('Bosnia and Herzegovina', 2),
 ('Australia', 2),
 ('France', 2),
 ('Croatia', 2),
 ('Argentina', 1)]

22/02/28 01:57:46 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 6768516 ms exceeds timeout 120000 ms
22/02/28 01:57:46 WARN SparkContext: Killing executors is not supported by current scheduler.
